In [ ]:
import requests
from bs4 import BeautifulSoup
import subprocess
import re
import os
import stanfordnlp
import pandas as pd

from ProcessArticle import ProcessArticle

from WikiArticleXmlHandler import WikiArticleXmlHandler
from WikiDataXmlHandler import WikiDataXmlHandler
import xml.sax
import mwparserfromhell

from timeit import default_timer as timer

from multiprocessing import Pool 
from multiprocessing import Lock
import tqdm 

# List of lists to single list
from itertools import chain

# Sending keyword arguments in map
from functools import partial

import gc

import warnings
warnings.filterwarnings('ignore')

In [ ]:
infoboxes = pd.read_csv("./2.data/infobox-list.csv")["infobox template name"].values.tolist()

article_folder = "./0.articles"
articles = [article_folder + "/" + x for x in os.listdir(article_folder)]
article = articles[0] # process only one file for now

wikidata_folder = "./1.wikidata"
wikidatas = [wikidata_folder + "/" + x for x in os.listdir(wikidata_folder)]
wikidata = wikidatas[0]

n_write = 1000

out_file = "./saved/time_test"

nlp = stanfordnlp.Pipeline(models_dir="./stanfordnlp_resources/", use_gpu=False);

func = ProcessArticle(infoboxes, nlp)

In [ ]:
if not os.path.exists(out_file + "_wikidata.csv"):
    pd.DataFrame(columns=["title", "label", "type"]).to_csv(
        out_file + "_wikidata.csv", header=True, index=False, sep="\t")

In [ ]:
%%time

handler = WikiDataXmlHandler(out_file + "_wikidata.csv")
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

print("Started processing: {}".format(wikidata))
for line in subprocess.Popen(["bzcat"],
                        stdin=open(wikidata),
                        stdout=subprocess.PIPE).stdout:
    try:
        parser.feed(line)

        if handler._ready and (handler._counter % n_write) == 0:
            handler.write_df()
    except StopIteration:
        break

# making sure that everything was written to the file
handler.write_df()

In [ ]:
%%time

handler = WikiArticleXmlHandler(out_file + ".csv", func)
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

print("Started processing: {}".format(article))
for line in subprocess.Popen(["bzcat"],
                        stdin=open(article),
                        stdout=subprocess.PIPE).stdout:
    try:
        parser.feed(line)

        if handler._ready and (handler._counter % 100) == 0:
            handler.write_df()
    except StopIteration:
        break

# making sure that everything was written to the file
handler.write_df()